Geojson2json.py is a simple script that directly converts GeoJSON files extracted using the GPlates GUI to the World Builder-compatible JSON format compatible with .wb . Currently, it supports continental plate geometries. 

The script reads the GeoJSON file, processes polygon features, and assigns default or provided values for 
maximum depth and compositions. The output JSON file follows the World Builder structure, making it ready 
for use in geodynamic modeling applications.

Future improvements may integrate this script with GPlate2GWB for a more streamlined workflow.

Usage:
    python geojson2json.py


In [ ]:
import json

# Configuration defined by the user
geojson_file = "reconstructed_400.00Ma.geojson"  # Input GeoJSON file
output_file = "world_builder_model.json"         # Output JSON file
default_max_depth = 350e3
default_compositions = [0]

try:
    # Load GeoJSON file
    with open(geojson_file, 'r') as f:
        geojson_data = json.load(f)

    features = []

    for feature in geojson_data.get('features', []):
        if feature.get('geometry', {}).get('type') == 'Polygon':
            name = feature.get('properties', {}).get('NAME', 'Unknown')
            coordinates = [list(coord) for coord in feature['geometry']['coordinates'][0]]

            # Use provided properties or fall back to defaults
            max_depth = feature.get('properties', {}).get('MAX_DEPTH', default_max_depth)
            compositions = feature.get('properties', {}).get('COMPOSITIONS', default_compositions)

            feature_data = {
                "model": "continental plate",
                "name": name,
                "max depth": max_depth,
                "coordinates": coordinates,
                "composition models": [
                    {
                        "model": "uniform",
                        "compositions": compositions,
                        "max depth": max_depth
                    }
                ]
            }
            features.append(feature_data)

    output_data = {
        "version": "1.1",
        "interpolation": "continuous monotone spline",
        "coordinate system": {
            "model": "spherical",
            "depth method": "starting point"
        },
        "features": features
    }

    # Save output to JSON file
    with open(output_file, 'w') as out_f:
        json.dump(output_data, out_f, indent=4)

    print(f"Conversion successful! Output saved to {output_file}")

except Exception as e:
    print(f"Error processing file: {e}")
